In [1]:
import numpy as np
import pandas as pd
import pickle
import ast
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Word Level/Sentence Tag idx.csv')

In [3]:
def fetch_file(path):
    file = open(path,'rb')
    a,b = pickle.load(file)
    file.close()
    return a,b

In [4]:
idx2word,word2idx =fetch_file('Word Level/word_map.pkl')
idx2tag,tag2idx = fetch_file('Word Level/tag_map.pkl')
df['Word idx'] =df['Word idx'].apply(lambda x: ast.literal_eval(x))
df['Tag idx'] =df['Tag idx'].apply(lambda x: ast.literal_eval(x))

In [5]:
X = df['Word idx'].to_list()
Y = df['Tag idx'].to_list()

In [6]:
X_train_d,X_test_d, y_train_d,y_test_d = train_test_split(X,Y, test_size = 0.33)

In [7]:
max_sen_len = max([len(i) for i in X])

In [8]:
X_train = word2idx['PAD']*np.ones((len(X_train_d),max_sen_len))
y_train = -1*np.ones((len(y_train_d), max_sen_len))

In [9]:
for j in range(len(X_train_d)):
    sen_len = len(X_train_d[j])
    temp_x = np.array(X_train_d[j])
    temp_y = np.array(y_train_d[j])
    X_train[j][:sen_len] = (temp_x)
    y_train[j][:sen_len] = (temp_y)

In [10]:
X_train, y_train = torch.LongTensor(X_train), torch.LongTensor(y_train)
X_train, y_train = Variable(X_train), Variable(y_train)

In [11]:
class Net(nn.Module):
    def __init__(self,vocab_size,embedding_dim,lstm_hidden_dim,number_of_tags):
        super(Net, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, batch_first = True)
        self.fc= nn.Linear(lstm_hidden_dim, number_of_tags)
    
    def forward(self, s):
        s = self.embedding(s)
        s, _ = self.lstm(s)
        s = s.view(-1, s.shape[2])
        s = self.fc(s)
        
        return F.log_softmax(s, dim = 1)
    
    def loss_fn(self,outputs, labels):
        labels = labels.view(-1)
        mask = (labels >= 0).float()
        #num_tokens = int(torch.sum(mask).data[0])
        outputs = outputs[range(outputs.shape[0]), labels]*mask
        
        return -torch.sum(outputs)/17          

In [12]:
vocab_size = len(word2idx)
embedding_dim = max_sen_len
lstm_hidden_dim = 64
num_tags = len(tag2idx)
model = Net(vocab_size = vocab_size + 1, embedding_dim = embedding_dim, lstm_hidden_dim = lstm_hidden_dim, number_of_tags = num_tags)

In [13]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [14]:
data_dataset = []
for i in range(len(X_train)):
    data_dataset.append((X_train[i],y_train[i]))

In [15]:
Data_train = torch.utils.data.DataLoader(data_dataset, shuffle = True)

In [16]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(Data_train):
        sent, labels = data
        optimizer.zero_grad()        
        
        output = model(sent)
        loss = model.loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 1.189
[1,  4000] loss: 0.923
[1,  6000] loss: 0.877
[1,  8000] loss: 0.831
[1, 10000] loss: 0.779
[1, 12000] loss: 0.741
[1, 14000] loss: 0.703
[1, 16000] loss: 0.664
[1, 18000] loss: 0.647
[1, 20000] loss: 0.632
[1, 22000] loss: 0.609
[1, 24000] loss: 0.587
[1, 26000] loss: 0.563
[1, 28000] loss: 0.543
[1, 30000] loss: 0.530
[1, 32000] loss: 0.525
[2,  2000] loss: 0.498
[2,  4000] loss: 0.489
[2,  6000] loss: 0.478
[2,  8000] loss: 0.479
[2, 10000] loss: 0.470
[2, 12000] loss: 0.453
[2, 14000] loss: 0.438
[2, 16000] loss: 0.444
[2, 18000] loss: 0.433
[2, 20000] loss: 0.433
[2, 22000] loss: 0.390
[2, 24000] loss: 0.408
[2, 26000] loss: 0.417
[2, 28000] loss: 0.410
[2, 30000] loss: 0.410
[2, 32000] loss: 0.398
Finished Training


In [24]:
torch.save(model,'Word Level/model.pytorch')